In [1]:
# # List all directories and files in the current working directory
# for root, dirs, files in os.walk('.'):
#     print("Root directory:", root)
#     print("Subdirectories:", dirs)
#     print("Files:", files)
#     break  # Stop after the first level to avoid printing too much

In [9]:
import os
os.chdir("../")    #commment after first run


In [10]:
#imports
import dask.distributed
import pystac_client
import planetary_computer
import stackstac 
import numpy as np
import pandas as pd
import rioxarray
import xarray as xr
import geopandas as gpd
from src.utils import gen_chips


In [11]:
print(os.getcwd())

/home/benchuser/code


In [12]:
#config setup
import yaml
with open("notebooks/config.yml", "r") as file:
    config = yaml.safe_load(file)
print(config)  # Check the structure of the dictionary


{'sentinel_2': {'collection': 'sentinel-2-l2a', 'time_ranges': ['2023-01-01/2023-03-31', '2023-04-01/2023-06-30', '2023-07-01/2023-09-30', '2023-10-01/2023-12-31'], 'cloud_cover': 1, 'bands': ['B2', 'B3', 'B4', 'B8', 'B11', 'B12'], 'resolution': 10}, 'land_cover': {'collection': 'io-lulc-annual-v02', 'year': '2023-01-01/2023-12-31'}, 'chips': {'sample_size': 256, 'chip_size': 128}, 'output': {'directory': 'notebooks/test_output_dump', 'naming_convention': 's2_{season}_{index:05}.tif'}, 'metadata': {'file': 'metadata.csv'}}


In [13]:
# Sentinel-2 settings
s2_collection = config["sentinel_2"]["collection"]
s2_date_ranges = config["sentinel_2"]["time_ranges"]
s2_bands = config["sentinel_2"]["bands"]
s2_resolution = config["sentinel_2"]["resolution"]
cloud_cover_threshold = config["sentinel_2"]["cloud_cover"]  # Max allowed cloud cover

# Land Cover settings
lc_collection = config["land_cover"]["collection"]
lc_year = config["land_cover"]["year"]  # Year of LC dataset

# Chip settings
sample_size = config["chips"]["sample_size"]  # Grid size for homogeneity check
chip_size = config["chips"]["chip_size"]  # Output chip size

# Output settings
output_dir = config["output"]["directory"]
chip_naming_convention = config["output"]["naming_convention"]

# Metadata settings
metadata_file = config["metadata"]["file"]

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define seasons for indexing
seasons = ["JFM", "AMJ", "JAS", "OND"]

aoi_gdf = gpd.read_file("data/urbans.geojson") # or "data/aois.geojson"


In [14]:
#dask
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(cluster)
print(client.dashboard_link)

http://127.0.0.1:8787/status


Task exception was never retrieved
future: <Task finished name='Task-299532' coro=<Client._gather.<locals>.wait() done, defined at /opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File "/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/client.py", line 2394, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-299533' coro=<Client._gather.<locals>.wait() done, defined at /opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File "/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/client.py", line 2394, in wait
    raise AllExit()
distributed.client.AllExit


In [26]:

def search_s2_scenes(aoi, date_range):
    print(f"Searching for Sentinel-2 scenes for AOI at {aoi.geometry.bounds} within {date_range}")
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
    )
    s2_search = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=aoi.geometry.bounds,
        datetime=date_range,
        query=["eo:cloud_cover<1"],
        sortby=["+properties.eo:cloud_cover"],
        max_items=1,
    )
    items = s2_search.item_collection()
    print(f"Found {len(items)} Sentinel-2 scenes")
    return items

def search_lc_scene(bbox, lc_date_range):
    print(f"Searching for Land Cover scenes within {lc_date_range} for bbox {bbox}")
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
    )
    lc_search = catalog.search(
        collections=["io-lulc-annual-v02"],
        bbox=bbox,
        datetime=lc_date_range,
    )
    items = lc_search.item_collection()
    print(f"Found {len(items)} Land Cover scenes")
    return items

def stack_s2_data(s2_items, s2_assets):
    if not s2_items:
        print("No Sentinel-2 items found.")
        return None
    
    try:
        print("Stacking Sentinel-2 images...")
        stacked_data = stackstac.stack(
            s2_items,
            assets=s2_assets,
            epsg=s2_items[0].properties["proj:epsg"],
            resolution=10,
            bounds_latlon=s2_items[0].bbox,
        ).median("time", skipna=True).squeeze()

        print("Stacked S2 data shape:", stacked_data.shape)

        if stacked_data is None or not isinstance(stacked_data, xr.DataArray):
            print("Error: Stacking Sentinel-2 data resulted in an invalid output.")
            return None

        return stacked_data

    except Exception as e:
        print(f"Error stacking Sentinel-2 data: {e}")
        return None

def stack_lc_data(lc_items, s2_epsg):
    if len(lc_items) == 0:
        print("No Land Cover data found.")
        return None
    try:
        print("Stacking Land Cover images...")
        stacked_data = stackstac.stack(
            lc_items,
            dtype=np.ubyte,
            fill_value=255,
            sortby_date=False,
            epsg=s2_epsg,
            resolution=10,
            bounds_latlon=lc_items[0].bbox,
        ).squeeze()
        print("Stacked LC data shape:", stacked_data.shape)
        return stacked_data
    except Exception as e:
        print(f"Error stacking Land Cover data: {e}")
        return None

def unique_class(window, axis=None, **kwargs):
    return np.all(window == window[:, :1, :, :1], axis=(1, 3))


def is_homogeneous(lc_stack):
    print("Checking for homogeneous LC regions...")

    # Ensure dimensions are large enough to process
    height, width = lc_stack.shape[-2], lc_stack.shape[-1]

    if height < sample_size or width < sample_size:
        print(f"AOI too small ({height}, {width}) for processing. Skipping this AOI.")
        return None  # Prevent infinite looping

    # Compute cropping dimensions
    new_height = max((height // sample_size) * sample_size, sample_size)
    new_width = max((width // sample_size) * sample_size, sample_size)

    print("old height and width:", height, width, "new height and width:", new_height, new_width)

    if new_height != height or new_width != width:
        print(f"Cropping LC stack to ({new_height}, {new_width}) to match sample_size.")
        lc_stack = lc_stack.isel(x=slice(0, new_height), y=slice(0, new_width))

    # Drop mismatched coordinate variables
    lc_stack = lc_stack.drop_vars([var for var in lc_stack.coords if "x" in var or "y" in var], errors="ignore")

    # Ensure LC stack is still valid before coarsening
    if lc_stack.shape[-2] < sample_size or lc_stack.shape[-1] < sample_size:
        print("Error: LC stack is too small after cropping. Skipping this AOI.")
        return None

    # Apply coarsening
    return lc_stack.coarsen(x=sample_size, y=sample_size, boundary="trim").reduce(unique_class)
    
def has_missing_values(array):
    print("Checking for missing values...")
    return array.isnull().any().compute()

def process_chips(aoi, s2_stack, lc_stack, output_dir, global_index):
    print(f"Processing chips for AOI at {aoi.geometry.bounds}")

    #  Compute homogeneity only ONCE before looping
    lc_uniqueness = is_homogeneous(lc_stack)
    if lc_uniqueness is None:
        print(f"Skipping AOI at {aoi.geometry.bounds}: No valid LC regions.")
        return global_index  # Skip processing if LC data is invalid

    for i in range(0, lc_stack.shape[1] - chip_size, sample_size):
        for j in range(0, lc_stack.shape[2] - chip_size, sample_size):
            if lc_uniqueness.isel(x=i//sample_size, y=j//sample_size) == False:
                print(f"Skipping chip at ({i}, {j}): LC region not homogeneous.")
                continue  
           
            s2_chip = s2_stack.isel(x=slice(i, i + chip_size), y=slice(j, j + chip_size))
            lc_chip = lc_stack.isel(x=slice(i, i + chip_size), y=slice(j, j + chip_size))

            if has_missing_values(s2_chip):
                print(f"Skipping chip at ({i}, {j}): Contains missing values.")
                continue  

            chip_name = f"s2_{date_range[:3]}_{global_index:05}.tif"
            lc_chip.rio.to_raster(os.path.join(output_dir, f"lc_{chip_name}"))
            s2_chip.rio.to_raster(os.path.join(output_dir, chip_name))
            print(f"Saved chip {chip_name}")
            global_index += 1

    return global_index


In [27]:
## Main processing loop
chip_dict = {}
global_index = 0
for index, aoi in aoi_gdf.iterrows():
    print(f"Processing AOI {index} at {aoi.geometry.bounds}")

    # Search for Sentinel-2 images
    s2_items = [search_s2_scenes(aoi, date_range) for date_range in s2_date_ranges]
    
    # Ensure all quarters have data
    if any(len(items) == 0 for items in s2_items):
        print(f"Skipping AOI {index}: Missing Sentinel-2 data for at least one quarter.")
        continue

    # Extract first valid Sentinel-2 item
    s2_first_item = next(iter(s2_items[0]), None)  # Ensure this exists
    if s2_first_item is None:
        print("Skipping AOI: No valid Sentinel-2 item found.")
        continue

    # Stack Sentinel-2 images
    s2_stack = stack_s2_data(
        [next(iter(item), None) for item in s2_items if item], s2_bands
    )

    # ✅ Ensure s2_stack is valid
    if s2_stack is None:
        print(f"Skipping AOI {index}: No valid Sentinel-2 data found.")
        continue

    if not hasattr(s2_stack, "rio") or s2_stack.rio.crs is None:
        print(f"Error: s2_stack is missing CRS information. Assigning manually.")
        s2_epsg = s2_first_item.properties["proj:epsg"]
        s2_stack.rio.write_crs(f"epsg:{s2_epsg}", inplace=True)

    print(f"Sentinel-2 CRS: {s2_stack.rio.crs}")

    # Search for Land Cover images using `s2_first_item.bbox`
    lc_items = search_lc_scene(s2_first_item.bbox, lc_year)
    
    if len(lc_items) == 0:
        print(f"Skipping AOI {index}: No valid LC data found.")
        continue

    # ✅ Now safe to extract EPSG and stack LC data
    lc_stack = stack_lc_data(lc_items, s2_stack.rio.crs.to_epsg())

    if lc_stack is None:
        print(f"Skipping AOI {index}: No valid LC data found.")
        continue

    # Process chips
    global_index, chip_dict = process_chips(aoi, s2_stack, lc_stack, output_dir, global_index)



Processing AOI 0 at (31.20416267326229, 30.02337142235983, 31.280433633102717, 30.060612342021983)
Searching for Sentinel-2 scenes for AOI at (31.20416267326229, 30.02337142235983, 31.280433633102717, 30.060612342021983) within 2023-01-01/2023-03-31
Found 1 Sentinel-2 scenes
Searching for Sentinel-2 scenes for AOI at (31.20416267326229, 30.02337142235983, 31.280433633102717, 30.060612342021983) within 2023-04-01/2023-06-30
Found 1 Sentinel-2 scenes
Searching for Sentinel-2 scenes for AOI at (31.20416267326229, 30.02337142235983, 31.280433633102717, 30.060612342021983) within 2023-07-01/2023-09-30
Found 1 Sentinel-2 scenes
Searching for Sentinel-2 scenes for AOI at (31.20416267326229, 30.02337142235983, 31.280433633102717, 30.060612342021983) within 2023-10-01/2023-12-31
Found 1 Sentinel-2 scenes
Stacking Sentinel-2 images...
Stacked S2 data shape: (2, 11272, 11273)
Error: s2_stack is missing CRS information. Assigning manually.
Sentinel-2 CRS: EPSG:32636
Searching for Land Cover scenes

/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


Found 2 Land Cover scenes
Stacking Land Cover images...
Stacked LC data shape: (2, 89417, 65754)
Processing chips for AOI at (31.20416267326229, 30.02337142235983, 31.280433633102717, 30.060612342021983)
Checking for homogeneous LC regions...
old height and width: 89417 65754 new height and width: 89344 65536
Cropping LC stack to (89344, 65536) to match sample_size.


/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


KeyboardInterrupt: 

In [15]:
#optional plotting code
import matplotlib.pyplot as plt

# Select the first stored location
if chip_dict:
    first_location = next(iter(chip_dict.keys()))  # Get the first (i, j) key
    first_four_chips = chip_dict[first_location]  # Get the corresponding chips

    fig, axes = plt.subplots(2, 4, figsize=(15, 6))  # 2 rows, 4 columns

    for idx, (s2_chip, lc_chip) in enumerate(first_four_chips):
        # Sentinel-2 RGB Visualization (assuming B4, B3, B2 are RGB)
        rgb_image = s2_chip.sel(band=[3, 2, 1]).transpose("y", "x", "band")  # Adjust band indices if needed
        axes[0, idx].imshow(rgb_image.compute().clip(0, 3000) / 3000)  # Normalize for better visualization
        axes[0, idx].set_title(f"Sentinel-2 Chip {idx+1}")
        axes[0, idx].axis("off")

        # Land Cover Visualization
        axes[1, idx].imshow(lc_chip.compute(), cmap="tab10")  # Color map for categorical data
        axes[1, idx].set_title(f"Land Cover Chip {idx+1}")
        axes[1, idx].axis("off")

    plt.tight_layout()
    plt.show()

In [16]:
print(chip_dict)

{}
